In [1]:
import pandas

In [38]:
baseCsv = pandas.read_csv('./crash_data.csv', sep=',', index_col=False, low_memory=False)

In [3]:
baseCsv

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,MCP3170003V,240000438,Montgomery County Police,Property Damage Crash,01/03/2024 02:55:00 PM,NaN,NaN,NaN,NaN,IN FRONT OF 18900 BIRDSEYE DR,...,0,No,No,2017,LEXUS,SUV,NO MISUSE,39.165005,-77.249310,"(39.16500483, -77.24931)"
1,MCP3254003K,230072050,Montgomery County Police,Injury Crash,12/16/2023 12:36:00 PM,Maryland (State),GERMANTOWN RD,County,MIDDLEBROOK RD,NaN,...,35,No,No,2010,TOYT,PRIUS,NO MISUSE,39.178776,-77.267190,"(39.17877577, -77.26718974)"
2,EJ7887003Q,230074270,Gaithersburg Police Depar,Injury Crash,12/29/2023 12:00:00 PM,Maryland (State),GREAT SENECA HWY,Municipality,KENTLANDS BLVD,NaN,...,35,No,No,2021,SUBARU,FORRESTER,NO MISUSE,39.123574,-77.231769,"(39.12357374, -77.231769)"
3,MCP2674004J,230064598,Montgomery County Police,Property Damage Crash,11/05/2023 09:07:00 PM,Maryland (State),WOODFIELD RD,County,GLENDALOUGH RD,NaN,...,40,No,No,2019,DODGE,CHARGER,NO MISUSE,39.211742,-77.171461,"(39.21174219, -77.17146065)"
4,MCP25280008,230067019,Montgomery County Police,Property Damage Crash,11/18/2023 12:40:00 AM,Maryland (State),CLARKSBURG RD,Maryland (State),CLARKSBURG RD,NaN,...,20,No,No,2014,NISSAN,ROGUE,NaN,39.228915,-77.289091,"(39.22891483, -77.28909117)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172100,DM8338000C,16010228,Takoma Park Police Depart,Property Damage Crash,03/01/2016 10:01:00 AM,Municipality,NEW HAMPSHIRE SERVICE LA,Municipality,PRINCE GEORGES AVE,NaN,...,25,No,No,2016,FORD,EXPLORER,NO MISUSE,38.972560,-76.997466,"(38.97255976, -76.99746609)"
172101,MCP1182001S,170519976,Montgomery County Police,Property Damage Crash,07/19/2017 02:22:00 PM,County,CEDAR LA,Maryland (State),OLD GEORGETOWN RD,NaN,...,35,No,No,2008,NISSAN,ALTIMA,NO MISUSE,39.004640,-77.108502,"(39.00464, -77.10850167)"
172102,MCP1453008X,200046217,Montgomery County Police,Property Damage Crash,11/23/2020 07:37:00 AM,Maryland (State),RIDGE RD,Unknown,LITTLE SENECA PKWY,NaN,...,40,No,No,2008,CHEVROLET,EXPRESS,NO MISUSE,39.228963,-77.236757,"(39.22896333, -77.23675667)"
172103,MCP2568000M,190056701,Montgomery County Police,Property Damage Crash,11/23/2019 11:23:00 PM,County,SHADY GROVE RD,Ramp,RAMP 9 FR IS 370 EB TO SHADY GROVE RD,NaN,...,35,No,No,2018,FORD,TRANSIT VAN,NO MISUSE,39.120440,-77.180047,"(39.12043995, -77.18004738)"


In [4]:
baseCsv.describe()

,Speed Limit,Vehicle Year,Latitude,Longitude
count,172105.000000,172105.000000,172105.000000,172105.000000
mean,32.549752,1966.306946,39.083119,-77.112343
std,11.059987,340.615468,0.072032,0.098571
min,0.000000,0.000000,37.720000,-79.486000
25%,25.000000,2006.000000,39.024417,-77.189327
50%,35.000000,2011.000000,39.074933,-77.105412
75%,40.000000,2015.000000,39.139742,-77.039592
max,75.000000,9999.000000,39.990414,-75.527708


In [5]:
baseCsv.isnull().sum()

Report Number                          0
Local Case Number                      0
Agency Name                            0
ACRS Report Type                       0
Crash Date/Time                        0
Route Type                         16973
Road Name                          15937
Cross-Street Type                  17006
Cross-Street Name                  15951
Off-Road Description              156170
Municipality                      152979
Related Non-Motorist              166642
Collision Type                       585
Weather                            13354
Surface Condition                  20118
Light                               1445
Traffic Control                    25469
Driver Substance Abuse             31324
Non-Motorist Substance Abuse      167788
Person ID                              0
Driver At Fault                        0
Injury Severity                        0
Circumstance                      140746
Driver Distracted By                   0
Drivers License 

In [6]:
baseCsv['Report Number'] = baseCsv['Report Number'].astype(str).str.strip()
baseCsv = baseCsv.drop_duplicates(subset=['Report Number'], keep='first').reset_index(drop=True)


In [7]:
baseCsv.isnull().sum()

Report Number                         0
Local Case Number                     0
Agency Name                           0
ACRS Report Type                      0
Crash Date/Time                       0
Route Type                        12702
Road Name                         12038
Cross-Street Type                 12721
Cross-Street Name                 12048
Off-Road Description              84818
Municipality                      86411
Related Non-Motorist              91566
Collision Type                      489
Weather                            7848
Surface Condition                 14338
Light                               800
Traffic Control                   16178
Driver Substance Abuse            17454
Non-Motorist Substance Abuse      92665
Person ID                             0
Driver At Fault                       0
Injury Severity                       0
Circumstance                      78729
Driver Distracted By                  0
Drivers License State              6891


In [40]:
baseCsv['Route Type'].fillna("Unknown", inplace=True)#unknown
baseCsv['Road Name'].fillna("UNKNOWN", inplace=True)
baseCsv['Cross-Street Type'].fillna("Unknown", inplace=True)
baseCsv['Cross-Street Name'].fillna("UNKNOWN", inplace=True)
baseCsv['Off-Road Description'].fillna("UNKNOWN", inplace=True)
baseCsv['Municipality'].fillna("N/A", inplace=True)
baseCsv['Related Non-Motorist'].fillna("N/A", inplace=True)
baseCsv['Collision Type'].fillna("UNKNOWN", inplace=True)
baseCsv['Weather'].fillna("N/A", inplace=True)
baseCsv['Surface Condition'].fillna("N/A", inplace=True)
baseCsv['Light'].fillna("N/A", inplace=True)
#df['Weather'].replace('unknown', 'missing', inplace=True)
baseCsv['Traffic Control'].fillna("N/A", inplace=True)

baseCsv['Driver Substance Abuse'].replace('UNKNOWN', 'N/A', inplace=True)
baseCsv['Driver Substance Abuse'].fillna("N/A", inplace=True)

baseCsv['Non-Motorist Substance Abuse'].fillna("N/A", inplace=True)
baseCsv['Driver Substance Abuse'].fillna("N/A", inplace=True)
baseCsv['Circumstance'].fillna("N/A", inplace=True)
baseCsv['Drivers License State'].fillna("N/A", inplace=True)
baseCsv['Vehicle Damage Extent'].fillna("N/A", inplace=True)
baseCsv['Vehicle First Impact Location'].fillna("UNKNOWN", inplace=True)
baseCsv['Vehicle Second Impact Location'].fillna("UNKNOWN", inplace=True)
baseCsv['Vehicle Body Type'].fillna("UNKNOWN", inplace=True)
baseCsv['Vehicle Movement'].fillna("UNKNOWN", inplace=True)
baseCsv['Vehicle Continuing Dir'].fillna("Unknown", inplace=True)
baseCsv['Vehicle Going Dir'].fillna("Unknown", inplace=True)
baseCsv['Vehicle Make'].fillna("UNKNOWN", inplace=True)
baseCsv['Vehicle Model'].fillna("UNKNOWN", inplace=True)
baseCsv['Equipment Problems'].fillna("UNKNOWN", inplace=True)

baseCsv['Driverless Vehicle'] = baseCsv['Driverless Vehicle'].str.lower()
baseCsv['Driverless Vehicle'] = baseCsv['Driverless Vehicle'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

baseCsv['Parked Vehicle'] = baseCsv['Parked Vehicle'].str.lower()
baseCsv['Parked Vehicle'] = baseCsv['Parked Vehicle'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

baseCsv['Driver Substance Abuse'] = baseCsv['Driver Substance Abuse'].str.lower()
baseCsv['Driver Substance Abuse'] = baseCsv['Driver Substance Abuse'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

baseCsv['Driver At Fault'] = baseCsv['Driver At Fault'].str.lower()
baseCsv['Driver At Fault'] = baseCsv['Driver At Fault'].replace({
    'yes': True,
    'no': False,
    'unknown': False
})

/tmp/ipykernel_189101/1442422662.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  baseCsv['Route Type'].fillna("Unknown", inplace=True)#unknown
/tmp/ipykernel_189101/1442422662.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [9]:
 baseCsv.isnull().sum()

Report Number                     0
Local Case Number                 0
Agency Name                       0
ACRS Report Type                  0
Crash Date/Time                   0
Route Type                        0
Road Name                         0
Cross-Street Type                 0
Cross-Street Name                 0
Off-Road Description              0
Municipality                      0
Related Non-Motorist              0
Collision Type                    0
Weather                           0
Surface Condition                 0
Light                             0
Traffic Control                   0
Driver Substance Abuse            0
Non-Motorist Substance Abuse      0
Person ID                         0
Driver At Fault                   0
Injury Severity                   0
Circumstance                      0
Driver Distracted By              0
Drivers License State             0
Vehicle ID                        0
Vehicle Damage Extent             0
Vehicle First Impact Locatio

In [44]:
# Group 1: Report and Case Information
df_report_case = baseCsv[['Report Number', 'Local Case Number', 'Agency Name', "ACRS Report Type", "Crash Date/Time", 'Municipality', 'Location']]
df_report_case.insert(0, 'ID', range(1, 1 + len(df_report_case)))

# Group 2: Road and Location Information 
df_road_location = baseCsv[['Route Type', 'Road Name', 'Cross-Street Type', 
                       'Cross-Street Name', 'Off-Road Description', 'Latitude', 'Longitude']]
df_road_location.insert(0, 'ID', range(1, 1 + len(df_road_location)))


# Group 3: Vehicle Information 
df_vehicle = baseCsv[['Vehicle ID', 'Vehicle Damage Extent', 
                 'Vehicle First Impact Location', 'Vehicle Second Impact Location', 
                 'Vehicle Body Type', 'Vehicle Movement', 'Vehicle Continuing Dir', 
                 'Vehicle Going Dir', 'Speed Limit', 'Driverless Vehicle', 
                 'Parked Vehicle', 'Vehicle Year', 'Vehicle Make', 'Vehicle Model', 
                 'Equipment Problems']]
df_vehicle.insert(0, 'ID', range(1, 1 + len(df_vehicle)))

# Group 4: Driver and Person Information
df_driver_person = baseCsv[['Person ID', 'Driver At Fault', 
                       'Driver Substance Abuse', 'Driver Distracted By', 
                       'Drivers License State', 'Non-Motorist Substance Abuse', 
                       'Related Non-Motorist']]
df_driver_person.insert(0, 'ID', range(1, 1 + len(df_driver_person)))


# Group 5: Incident Details 
df_incident_details = baseCsv[['Collision Type', 'Weather', 'Surface Condition', 
                          'Light', 'Traffic Control', 'Injury Severity', 'Circumstance']]
df_incident_details.insert(0, 'ID', range(1, 1 + len(df_incident_details)))


In [46]:
df_report_case.to_csv('df_report_case.csv', index=False)
df_road_location.to_csv('df_road_location.csv', index=False)
df_vehicle.to_csv('df_vehicle.csv', index=False)
df_driver_person.to_csv('df_driver_person.txt', sep=',', index=False, header=True)
df_incident_details.to_csv('df_incident_details.txt', sep=',', index=False, header=True)

In [50]:
df_incident_details

,ID,Collision Type,Weather,Surface Condition,Light,Traffic Control,Injury Severity,Circumstance
0,1,OPPOSITE DIRECTION SIDESWIPE,CLOUDY,N/A,DAYLIGHT,NO CONTROLS,NO APPARENT INJURY,N/A
1,2,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NO APPARENT INJURY,N/A
2,3,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,SUSPECTED MINOR INJURY,N/A
3,4,SINGLE VEHICLE,CLEAR,DRY,DARK LIGHTS ON,NO CONTROLS,NO APPARENT INJURY,"ANIMAL, N/A"
4,5,SINGLE VEHICLE,RAINING,WET,DARK LIGHTS ON,YIELD SIGN,NO APPARENT INJURY,"RAIN, SNOW, WET"
...,...,...,...,...,...,...,...,...
172100,172101,OTHER,CLEAR,DRY,DAYLIGHT,N/A,NO APPARENT INJURY,N/A
172101,172102,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NO APPARENT INJURY,N/A
172102,172103,SAME DIR REAR END,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NO APPARENT INJURY,N/A
172103,172104,ANGLE MEETS LEFT TURN,RAINING,WET,DARK LIGHTS ON,TRAFFIC SIGNAL,NO APPARENT INJURY,"RAIN, SNOW, WET"


In [13]:
baseCsv['Vehicle ID'].is_unique

True